<a href="https://colab.research.google.com/github/BalthazazarDuc/Data-Science-Machine-Learning/blob/Sequential_BERT/ABC_Distilbert_sequence_classification_advanced.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Import and dataset loading

In [ ]:
!pip install transformers torch
!pip install accelerate
!pip install transformers[torch] accelerate --upgrade


  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (99 kB)
  Using cached nvidia_nvjitlink_cu12-12.4.127-py3-none-m

In [ ]:
import pandas as pd
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, Trainer, TrainingArguments
import torch
from torch.utils.data import Dataset
import accelerate
import transformers
print(accelerate.__version__)

0.29.3


In [ ]:
train=pd.read_csv('https://github.com/BalthazazarDuc/Data-Science-Machine-Learning/raw/main/train_letter.csv')
test=pd.read_csv('https://raw.githubusercontent.com/BalthazazarDuc/Data-Science-Machine-Learning/main/unlabelled_test_data.csv?token=GHSAT0AAAAAACOZ4C73AOLJGUSBJSLVF63OZQ72D6A')


In [ ]:
label_dict = {
    'A': 0,
    'B': 1,
    'C': 2
}

# Appliquer le mapping sur la colonne des étiquettes
train['difficulty'] = train['difficulty'].map(label_dict)

#Model setting

In [ ]:
# Charger le tokenizer
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-multilingual-cased')


# Charger le modèle pré-entraîné
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-multilingual-cased', num_labels=6)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.96M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/466 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/542M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
# Fonction pour tokeniser les données
def encode(texts):
    return tokenizer(texts.tolist(), max_length=256, padding="max_length", truncation=True)

# Application de la fonction de tokenisation
train_encodings = encode(train['sentence'])
test_encodings = encode(test['sentence'])

In [ ]:
class FrenchLevelDataset(Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels is not None:
            item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

    def __len__(self):
        return len(self.encodings['input_ids'])

# Création des instances de dataset
train_dataset = FrenchLevelDataset(train_encodings, train['difficulty'])
test_dataset = FrenchLevelDataset(test_encodings)


In [ ]:
# Charger le modèle pré-entraîné
model = DistilBertForSequenceClassification.from_pretrained('distilbert-base-multilingual-cased', num_labels=6)  # Assurez-vous que le nombre de labels correspond au nombre de niveaux de difficulté

# Configuration des paramètres d'entraînement

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch"
)

# Création de l'objet Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset  # Vous pouvez utiliser une partie des données train pour la validation, si nécessaire
)

# Lancement de l'entraînement
trainer.train()

# Évaluation
trainer.evaluate()

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Step,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
#Arguments for longer training (almost 4h)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=5,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    learning_rate=5e-5,
    warmup_steps=1000,
    weight_decay=0.05,
    logging_dir='./logs',
    logging_steps=10,
    evaluation_strategy="epoch"
)

In [ ]:
# Generate predictions for the test dataset
predictions = trainer.predict(test_dataset)
predicted_labels_indices = predictions.predictions.argmax(-1)  # Get the index of the max logit which is the predicted class


In [ ]:
# Assuming the label dictionary used earlier was like this:
label_dict = {'A': 0, 'B': 1, 'C': 2}
# Invert the dictionary to map back from indices to labels
index_to_label = {v: k for k, v in label_dict.items()}

# Map the predicted label indices to actual labels
predicted_labels = [index_to_label[idx] for idx in predicted_labels_indices]


In [ ]:
import pandas as pd

# Assuming the test DataFrame is named 'test' and has a column 'sentence' for the input text
test['difficulty'] = predicted_labels
results_df = test[['id', 'difficulty']]


In [ ]:
results_df.to_csv('predicted_results.csv', index=False)

In [ ]:
results_df

,id,difficulty
0,0,C2
1,1,B1
2,2,B1
3,3,A2
4,4,C2
...,...,...
1195,1195,B1
1196,1196,A2
1197,1197,C2
1198,1198,B2
